In [1]:
import numpy as np
import nibabel as nib
%matplotlib inline
import matplotlib.pyplot as plt
import albumentations as ab
from scipy.ndimage.filters import gaussian_filter, sobel
import skimage.transform as skTrans

In [2]:
def load_image(file_path):
    img = nib.load(file_path)
    return img.get_data()

def show_image(image, mask, index=0):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize = (12, 6))
    ax1.imshow(image[:, index, :], cmap='gray', vmin=0.0, vmax=1.0)
    ax1.set_title('Original')
    ax2.imshow(mask[:, index, :], cmap='gray', vmin=0.0, vmax=1.0)
    ax2.set_title('Augmented')

def do_crop_3d(image, x, y, z, h, w, d):

    return image[x:x + h, y:y + w, z:z + d, :]

def adjust_brightness(image, delta=0.125):

    return image + delta

def adjust_contrast(image, contrast_factor = 1.5):
    image_mean = np.mean(image)

    return (contrast_factor * (image - image_mean)) + image_mean

def rotate(image):
    return np.transpose(np.flip(image, 1), (1, 0, 2, 3))  # 90 deg Z


def resize(image, new_size):
    # image.shape[3] is the number of channels
    new_image = np.zeros((new_size, new_size, new_size, image.shape[3]))

    for z in range(image.shape[2]):
            new_image[:, :, z, :] = np.array(ab.Resize(new_size, new_size)(image=image[:,:,z])["image"])
    return new_image

def crop_and_resize(image, alpha=4):

    # All sides have the same size so anyone would do
    max_crop_length = int(image.shape[0] / alpha)
    max_start = (image.shape[0] - max_crop_length) - 2 # To make sure no IndexOutOfBound occurs

    # Select the starting point of the cropping randomly
    start_x = np.random.randint(0, max_start)
    start_y = np.random.randint(0, max_start)
    start_z = np.random.randint(0, max_start)

    # Cropp a cubic image, if another form of cropping is required then we need to pass different 
    # values instead of the same max_crop_length for all
    cropped_image = do_crop_3d(image, start_x, start_y, start_z, max_crop_length, max_crop_length, max_crop_length)
    
    return resize(cropped_image, image.shape[0])
    #return skTrans.resize(cropped_image, image.shape, order=1, preserve_range=True)

def add_gaussian_noise(image, max_mean=0.3, max_sigma=0.1):
    mean = np.random.uniform(-max_mean, max_mean)
    sigma = np.random.uniform(0.0, max_sigma)
    
    gauss = np.random.normal(mean, sigma,image.shape)
    
    return image + gauss

def apply_gaussian_filter(image, max_sigma=1.0):
    sigma = np.random.uniform(0.0, max_sigma)
    
    return gaussian_filter(image, sigma)

def apply_sobel_filter(image):
    dx = sobel(image, 0)
    dy = sobel(image, 1)
    dz = sobel(image, 2)
    
    mag = np.sqrt(dx**2 + dy**2 + dz**2)
    filtered  = mag * (1.0 / np.max(mag))
    
    return filtered

def cut_out(image, alpha=4):

    # All sides have the same size so anyone would do
    max_cutout_length = int(image.shape[0] / alpha)
    max_start = (image.shape[0] - max_cutout_length) - 2 # To make sure no IndexOutOfBound occurs

    # Select the starting point of the cropping randomly
    start_x = np.random.randint(0, max_start)
    end_x = start_x +  max_cutout_length
    
    start_y = np.random.randint(0, max_start)
    end_y = start_y + max_cutout_length
    
    start_z = np.random.randint(0, max_start)
    end_z = start_z +  max_cutout_length
    
    cutout_pixels = np.zeros((end_x - start_x, end_y - start_y, end_z - start_z,1))
    
    cutout_image = image.copy()
    cutout_image[start_x:end_x, start_y:end_y, start_z:end_z, :] =  cutout_pixels
    
    return cutout_image
    


In [19]:
#image = load_image("pancreas_001.nii.gz")
#image = load_image("my_image.nii")
image = np.load("/Users/d070867/netstore/workspace/cpc_pancreas3d/Task07_Pancreas/Task07_Pancreas/result/test_labels/pancreas_015_9_label.npy")
ee = np.eye(3)
print(ee)
print(image.shape)
image = np.rint(image).astype(np.int)
image = ee[image]
#print()
print(image.shape)
#augmented = adjust_brightness(image)
#augmented = adjust_contrast(image)
#augmented = rotate(image)
#augmented = crop_and_resize(image)
#augmented = add_gaussian_noise(image)
#augmented = apply_gaussian_filter(image)
#augmented = apply_sobel_filter(image)
#augmented = cut_out(image)

#show_image(image, augmented, 35)
#show_image(image, augmented, 0)
#show_image(image, augmented, 10)
#show_image(image, augmented, 10)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
(64, 64, 64, 1)
(64, 64, 64, 1, 3)
